<a href="https://colab.research.google.com/github/chw8207/NLP-study/blob/main/%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [51]:
# 사전 훈련된 단어 임베딩 사용하기
class PreTrainedEmbeddings(object) : 
  def __init__(self, word_to_index, word_vectors) : 
      """
        매개변수:
            word_to_index (dict): 단어에서 정수로 매핑
            word_vectors (numpy 배열의 리스트)
      """
      self.word_to_index = word_to_index
      self.word_vectors = word_vectors
      self.index_to_word = {v:k for k, v in self.word_to_index.items()}
      # AnnoyIndex(len(word_vectors[0]), metric='거리척도')
      # 근사 최근접 이웃 탐색을 제공함
      # len(word_vectors[0]) : 첫 번째 단어 벡터의 길이 활용 
      #                        모든 단어의 벡터 길이가 동일해야하기 때문
      self.index = AnnoyIndex(f=100, metric='euclidean') 
      print("인덱스 만드는 중!")
      # i = 0
      for _,i in word_to_index.items() : 
        self.index.add_item(i, word_vectors[i])
      self.index.build(50)
      print("완료!")

  @classmethod
  def from_embeddings_file(cls, embedding_file) : 
    """사전 훈련된 벡터 파일에서 객체를 만듭니다.
        
        벡터 파일은 다음과 같은 포맷입니다:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        매개변수:
            embedding_file (str): 파일 위치
        반환값:
            PretrainedEmbeddings의 인스턴스
    """
    word_to_index = {}
    word_vectors = []
    with open(embedding_file) as fp : 
      for line in fp.readlines() : 
        line = line.split(' ')
        word = line[0]
        # 리스트의 나머지 요소들을 실수로 변환 후
        # 넘파이 배열로 변환 후 벡터를 생성함
        vec = np.array([float(x) for x in line[1:]])
        # 해당 단어가 주어지면 이 단어를 딕셔너리의 키로 사용
        # 딕셔너리의 현재 값을 해당 단어의 인덱스로 설정
        word_to_index[word] = len(word_to_index)
        word_vectors.append(vec)
    return cls(word_to_index, word_vectors)

# 단어 임베딩을 활용한 유추 작업
  def get_embedding(self, word) : 
    """
        매개변수:
            word (str)
        반환값
            임베딩 (numpy.ndarray)
    """
    return self.word_vectors[self.word_to_index[word]]

  def get_closets_to_vector(self, vector, n=1) : 
    """벡터가 주어지면 n 개의 최근접 이웃을 반환합니다
        매개변수:
            vector (np.ndarray): Annoy 인덱스에 있는 벡터의 크기와 같아야 합니다
            n (int): 반환될 이웃의 개수
        반환값:
            [str, str, ...]: 주어진 벡터와 가장 가까운 단어
                단어는 거리순으로 정렬되어 있지 않습니다.
    """
    nn_indices = self.index.get_nns_by_vector(vector, n)
    return [self.index_to_word[neighbor] for neighbor in nn_indices]

  def compute_and_print_analogy(self, word1, word2, word3) : 
    """단어 임베딩을 사용한 유추 결과를 출력합니다

        word1이 word2일 때 word3은 __입니다.
        이 메서드는 word1 : word2 :: word3 : word4를 출력합니다
        
        매개변수:
            word1 (str)
            word2 (str)
            word3 (str)
    """
    vec1 = self.get_embedding(word1)
    vec2 = self.get_embedding(word2)
    vec3 = self.get_embedding(word3)

    # 네 번째 단어 임베딩 계산
    # 단순 가정 : 이 유추는 공간적 관계(spatial_relationship)
    spatial_relationship = vec2 - vec1
    vec4 = vec3 + spatial_relationship

    closet_words = self.get_closets_to_vector(vec4, n=4)
    existing_words = set([word1, word2, word3])
    closet_words = [word for word in closet_words if word not in existing_words]

    if len(closet_words) == 0 :
      print('계산된 벡터와 가장 가까운 이웃을 찾을 수 없습니다.')
      return

    for word4 in closet_words : 
      print(f'{word1} : {word2} :: {word3} : {word4}')

In [31]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-06-09 05:06:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-06-09 05:06:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-06-09 05:06:58--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [52]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/glove/glove.6B.100d.txt')

인덱스 만드는 중!
완료!


In [53]:
# 관계 1 : 성별 명사와 대명사의 관계
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : never


In [54]:
# 동사-명사 관계
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel


In [55]:
# 상위어(Hypernymy)(더 넓은 범주)
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

blue : color :: dog : animal
blue : color :: dog : breed
blue : color :: dog : pet


In [56]:
# 부분에서 전체(Meronymy)
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : attached
toe : foot :: finger : apart


In [57]:
# 비교급
embeddings.compute_and_print_analogy('fast', 'fastest', 'young')

fast : fastest :: young : youngest
fast : fastest :: young : sixth
fast : fastest :: young : fifth
fast : fastest :: young : third


In [58]:
# 문화적 성별 편견
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician
